In [1]:
import os
import json
import pandas as pd
import traceback

In [ ]:
pip install --upgrade langchain

In [9]:
from langchain_openai import ChatOpenAI

In [11]:
from dotenv import load_dotenv

load_dotenv() # take environment variables from .env

True

In [12]:
KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)